# AI Pair Programming

Dans ce notebook nous allons demander à une intelligence artificielle de générer du code Python pour réaliser des tâches de traitement automatique de corpus.

Avant de commencer, choisissez un outil comme [Bard](https://bard.google.com/u/2/chat) ou [ChatGPT](https://chat.openai.com/) et créez un compte.

Vous pouvez ensuite demander à l'outil de créer du code. Avant de commencer, n'hésitez pas à lire [cet article](https://exocoding.com/ai-code-generation/) qui détaille les **bonnes pratiques** pour créer des _prompts_ efficaces dans le cadre de la génération de code par l'intelligence artificielle.


### 1. Algorithme simple en Python

Demandez à l'IA de générer un code python qui lance un décompte du réveillon du Nouvel An. Le code doit imprimer les nombres de 10 à 0 avec un intervalle d'une seconde, puis imprimer "Bonne année" à la fin.

In [1]:
import time

print("Décompte du Nouvel An !\n")

for i in range(10, 0, -1):
    print(i)
    time.sleep(1)

print("0")
time.sleep(1)
print("\n🎉 Bonne année ! 🎉")

Décompte du Nouvel An !

10
9
8
7
6
5
4
3
2
1
0

🎉 Bonne année ! 🎉


### 2. Détection du sujet de la phrase

Demandez à l'IA d'extraire le sujet dans une phrase.
Demandez ensuite de générer le code Python qui réalise cette tâche et testez le ci-dessous.

In [ ]:
import spacy

# Charger le modèle français de spaCy
# Installation requise: python -m spacy download fr_core_news_sm
try:
    nlp = spacy.load("fr_core_news_sm")
except:
    print("Erreur: Le modèle français n'est pas installé.")
    print("Installez-le avec: python -m spacy download fr_core_news_sm")
    exit()

def extraire_sujet(phrase):
    """
    Extrait le sujet d'une phrase en français.
    """
    doc = nlp(phrase)
    
    sujets = []
    for token in doc:
        # Chercher les tokens avec la dépendance 'nsubj' (sujet nominal)
        if token.dep_ in ['nsubj', 'nsubj:pass']:
            # Récupérer le sujet complet avec ses modificateurs
            sujet_complet = []
            for child in token.subtree:
                sujet_complet.append(child.text)
            sujets.append(' '.join(sujet_complet))
    
    return sujets if sujets else ["Aucun sujet trouvé"]

# Tests
phrases_test = [
    "Le chat dort sur le canapé.",
    "Marie et Pierre vont au cinéma.",
    "Les étudiants travaillent dur pour leurs examens.",
    "Mon petit frère joue dans le jardin.",
    "La voiture rouge est garée devant la maison."
]

print("=== Extraction de sujets ===\n")
for phrase in phrases_test:
    sujet = extraire_sujet(phrase)
    print(f"Phrase: {phrase}")
    print(f"Sujet(s): {', '.join(sujet)}")
    print("-" * 50)

# Test interactif
print("\n=== Test personnalisé ===")
phrase_utilisateur = input("Entrez une phrase: ")
sujet = extraire_sujet(phrase_utilisateur)
print(f"Sujet(s) trouvé(s): {', '.join(sujet)}")


=== Extraction de sujets ===

Phrase: Le chat dort sur le canapé.
Sujet(s): Aucun sujet trouvé
--------------------------------------------------
Phrase: Marie et Pierre vont au cinéma.
Sujet(s): Marie et Pierre
--------------------------------------------------
Phrase: Les étudiants travaillent dur pour leurs examens.
Sujet(s): Les étudiants
--------------------------------------------------
Phrase: Mon petit frère joue dans le jardin.
Sujet(s): Mon petit frère
--------------------------------------------------
Phrase: La voiture rouge est garée devant la maison.
Sujet(s): La voiture, La voiture rouge
--------------------------------------------------

=== Test personnalisé ===


### 3. Entités nommmées liées à Wikidata

Demandez à l'IA d'extraire les entités nommées d'un texte en français, et de les lier à un identifiant wikidata.
Demandez ensuite à l'IA de générer le code Python pour réaliser cette tâche et testez le ci-dessous.

In [ ]:
import spacy
import requests
from urllib.parse import quote

# Charger le modèle français de spaCy
try:
    nlp = spacy.load("fr_core_news_sm")
except:
    print("Erreur: Le modèle français n'est pas installé.")
    print("Installez-le avec: python -m spacy download fr_core_news_sm")
    exit()

def rechercher_wikidata(entite, langue='fr'):
    """
    Recherche une entité sur Wikidata et retourne son identifiant.
    """
    url = "https://www.wikidata.org/w/api.php"
    params = {
        'action': 'wbsearchentities',
        'format': 'json',
        'language': langue,
        'search': entite,
        'limit': 1
    }
    
    try:
        response = requests.get(url, params=params, timeout=5)
        data = response.json()
        
        if data.get('search') and len(data['search']) > 0:
            resultat = data['search'][0]
            return {
                'id': resultat.get('id'),
                'label': resultat.get('label'),
                'description': resultat.get('description', 'Pas de description'),
                'url': f"https://www.wikidata.org/wiki/{resultat.get('id')}"
            }
    except Exception as e:
        print(f"Erreur lors de la recherche Wikidata: {e}")
    
    return None

def extraire_entites_avec_wikidata(texte):
    """
    Extrait les entités nommées d'un texte et les lie à Wikidata.
    """
    doc = nlp(texte)
    entites_trouvees = []
    
    for ent in doc.ents:
        info_wikidata = rechercher_wikidata(ent.text)
        
        entite_info = {
            'texte': ent.text,
            'type': ent.label_,
            'wikidata': info_wikidata
        }
        entites_trouvees.append(entite_info)
    
    return entites_trouvees

def afficher_resultats(entites):
    """
    Affiche les résultats de manière formatée.
    """
    print(f"\n{'='*80}")
    print(f"Nombre d'entités trouvées: {len(entites)}")
    print(f"{'='*80}\n")
    
    for i, entite in enumerate(entites, 1):
        print(f"{i}. Entité: {entite['texte']}")
        print(f"   Type: {entite['type']}")
        
        if entite['wikidata']:
            wd = entite['wikidata']
            print(f"   Wikidata ID: {wd['id']}")
            print(f"   Label: {wd['label']}")
            print(f"   Description: {wd['description']}")
            print(f"   URL: {wd['url']}")
        else:
            print(f"   Wikidata: Non trouvé")
        
        print(f"{'-'*80}\n")

# Textes de test
textes_test = [
    """
    Emmanuel Macron est le président de la France. Il habite à Paris, 
    au Palais de l'Élysée. Il a rencontré Joe Biden à Washington.
    """,
    """
    Albert Einstein a développé la théorie de la relativité. 
    Il a travaillé à l'Université de Princeton aux États-Unis.
    """,
    """
    La Tour Eiffel est située à Paris. Elle a été construite par 
    Gustave Eiffel pour l'Exposition universelle de 1889.
    """
]

# Tester avec les textes prédéfinis
print("=== TEST AUTOMATIQUE ===\n")
for i, texte in enumerate(textes_test, 1):
    print(f"\n>>> TEXTE {i}:")
    print(texte.strip())
    entites = extraire_entites_avec_wikidata(texte)
    afficher_resultats(entites)

# Test interactif
print("\n" + "="*80)
print("=== TEST PERSONNALISÉ ===")
print("="*80)
texte_utilisateur = input("\nEntrez un texte à analyser (ou appuyez sur Entrée pour passer): ")

if texte_utilisateur.strip():
    print(f"\nAnalyse du texte: {texte_utilisateur}")
    entites = extraire_entites_avec_wikidata(texte_utilisateur)
    afficher_resultats(entites)
else:
    print("\nAucun texte saisi. Programme terminé.")

Erreur: Le modèle français n'est pas installé.
Installez-le avec: python -m spacy download fr_core_news_sm
=== TEST AUTOMATIQUE ===


>>> TEXTE 1:
Emmanuel Macron est le président de la France. Il habite à Paris, 
    au Palais de l'Élysée. Il a rencontré Joe Biden à Washington.


NameError: name 'nlp' is not defined

: 

### 4. A vous de jouer

Pensez à une analyse que vous voudriez faire sur un texte. Demandez à l'IA de générer un code python qui réalise cette analyse et testez le ci-dessous

In [5]:
# Votre code ici


### Pour aller plus loin...

En tant qu'étudiant ULB vous avez accès gratuitement au [Github student pack](https://education.github.com/pack). Ce pack vous donne permet d'utiliser [Github copilot](https://github.com/features/copilot), un outil d'auto-complétion de code grâce à l'intelligence artificielle. Ceci peut être très utile si vous voulez réaliser des tâches complexes sans être un expert en python.